# Lab | LangChain Med

## Objectives

- continue on with lesson 2' example, use different datasets to test what we did in class. Some datasets are suggested in the notebook but feel free to scout other datasets on HuggingFace or Kaggle.
- Find another model on Hugging Face and compare it.
- Modify the prompt to fit your selected dataset.

In [1]:
import numpy as np
import pandas as pd

## Load the Dataset
As you can see the notebook is ready to work with three different Datasets. Just uncomment the lines of the Dataset you want to use.

I selected Datasets with News. Two of them have just a brief decription of the news, but the other contains the full text.

As we are working in a free and limited space, I limited the number of news to use with the variable MAX_NEWS. Feel free to pull more if you have memory available.

The name of the field containing the text of the new is stored in the variable *DOCUMENT* and the metadata in *TOPIC*

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kotartemiy/topic-labeled-news-dataset")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'topic-labeled-news-dataset' dataset.
Path to dataset files: /kaggle/input/topic-labeled-news-dataset


In [3]:

news = pd.read_csv('/root/.cache/kagglehub/datasets/kotartemiy/topic-labeled-news-dataset/versions/1/labelled_newscatcher_dataset.csv', sep=';')
MAX_NEWS = 1000
DOCUMENT="title"
TOPIC="topic"

#news = pd.read_csv('/kaggle/input/bbc-news/bbc_news.csv')
#MAX_NEWS = 1000
#DOCUMENT="description"
#TOPIC="title"

#news = pd.read_csv('/kaggle/input/mit-ai-news-published-till-2023/articles.csv')
#MAX_NEWS = 100
#DOCUMENT="Article Body"
#TOPIC="Article Header"



ChromaDB requires that the data has a unique identifier. We can make it with this statement, which will create a new column called **Id**.


In [4]:
news["id"] = news.index
news.head()

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4


In [5]:
#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

## Import and configure the Vector Database
I'm going to use ChromaDB, the most popular OpenSource embedding Database.

First we need to import ChromaDB, and after that import the **Settings** class from **chromadb.config** module. This class allows us to change the setting for the ChromaDB system, and customize its behavior.

In [6]:
!pip install chromadb

In [7]:
import chromadb
from chromadb.config import Settings

Now we need to create the seetings object calling the **Settings** function imported previously. We store the object in the variable **settings_chroma**.

Is necessary to inform two parameters
* chroma_db_impl. Here we specify the database implementation and the format how store the data. I choose ***duckdb***, because his high-performace. It operate primarly in memory. And is fully compatible with SQL. The store format ***parquet*** is good for tabular data. With good compression rates and performance.

* persist_directory: It just contains the directory where the data will be stored. Is possible work without a directory and the data will be stored in memory without persistece, but Kaggle dosn't support that.

In [8]:
chroma_client = chromadb.PersistentClient(path="/chroma_client")
settings_chroma = Settings(chroma_db_impl="duckdb+parquet")

## Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection exist we need to delete it.

In the next lines, we are creating the collection by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [9]:
collection_name = "news_collection"
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)


It's time to add the data to the collection. Using the function ***add*** we need to inform, at least ***documents***, ***metadatas*** and ***ids***.
* In the **document** we store the big text, it's a different column in each Dataset.
* In **metadatas**, we can informa a list of topics.
* In **id** we need to inform an unique identificator for each row. It MUST be unique! I'm creating the ID using the range of MAX_NEWS.


In [10]:

collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 101MiB/s]


In [11]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)

{'ids': [['id173', 'id829', 'id117', 'id535', 'id141', 'id218', 'id390', 'id273', 'id56', 'id900']], 'embeddings': None, 'documents': [['The Legendary Toshiba is Officially Done With Making Laptops', '3 gaming laptop deals you can’t afford to miss today', 'Lenovo and HP control half of the global laptop market', 'Asus ROG Zephyrus G14 gaming laptop announced in India', 'Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865)', "Apple's Next MacBook Could Be the Cheapest in Company's History", "Features of Huawei's Desktop Computer Revealed", 'Redmi to launch its first gaming laptop on August 14: Here are all the details', 'Toshiba shuts the lid on laptops after 35 years', 'This is the cheapest Windows PC by a mile and it even has a spare SSD slot']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'

## Vector MAP

In [12]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [13]:

getado = collection.get(ids="id141",
                       include=["documents", "embeddings"])


In [14]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

array([[-8.08560848e-02, -4.99637052e-02, -2.37774849e-02,
        -1.10536022e-02,  2.66577117e-02, -4.47933301e-02,
        -2.88966335e-02,  2.66561043e-02,  1.43972272e-03,
        -1.64078418e-02,  6.53492734e-02, -6.90199286e-02,
        -5.74807823e-02,  1.01116151e-02,  5.04303500e-02,
        -2.05776445e-03,  7.25640804e-02, -1.24373689e-01,
         1.06594423e-02, -1.09420463e-01, -1.14324046e-02,
        -1.03760120e-02, -2.06108317e-02, -2.43940949e-02,
         7.82847628e-02,  5.82055887e-03,  2.33177263e-02,
        -8.24382976e-02, -2.72650588e-02,  4.66747722e-03,
         4.34018858e-03,  3.25280502e-02, -2.60309745e-02,
         7.96390548e-02,  4.21820618e-02, -1.21199943e-01,
         4.90708388e-02, -7.62584656e-02,  4.33162451e-02,
        -8.36045742e-02, -7.14040175e-02, -1.87925138e-02,
         3.60493883e-02,  4.28456254e-02,  2.57604383e-02,
         3.97251472e-02, -7.09129497e-03,  3.51899490e-02,
         2.73690969e-02,  9.28983930e-03, -3.91616635e-0

Once we have our information inside the Database we can query It, and ask for data that matches our needs. The search is done inside the content of the document, and it dosn't look for the exact word, or phrase. The results will be based on the similarity between the search terms and the content of documents.

The metadata is not used in the search, but they can be utilized for filtering or refining the results after the initial search.


## Loading the model and creating the prompt
TRANSFORMERS!!
Time to use the library **transformers**, the most famous library from [hugging face](https://huggingface.co/) for working with language models.

We are importing:
* **Autotokenizer**: It is a utility class for tokenizing text inputs that are compatible with various pre-trained language models.
* **AutoModelForCasualLLM**: it provides an interface to pre-trained language models specifically designed for language generation tasks using causal language modeling (e.g., GPT models), or the model used in this notebook ***databricks/dolly-v2-3b***.
* **pipeline**: provides a simple interface for performing various natural language processing (NLP) tasks, such as text generation (our case) or text classification.

The model selected is [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b), the smallest Dolly model. It have 3billion paramaters, more than enough for our sample, and works much better than GPT2.

Please, feel free to test [different Models](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending), you need to search for NLP models trained for text-generation. My recomendation is choose "small" models, or we will run out of memory in kaggle.  


In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)



tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

The next step is to initialize the pipeline using the objects created above.

The model's response is limited to 256 tokens, for this project I'm not interested in a longer response, but it can easily be extended to whatever length you want.

Setting ***device_map*** to ***auto*** we are instructing the model to automaticaly select the most appropiate device: CPU or GPU for processing the text generation.  

In [16]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",
)

Device set to use cpu


## Creating the extended prompt
To create the prompt we use the result from query the Vector Database  and the sentence introduced by the user.

The prompt have two parts, the **relevant context** that is the information recovered from the database and the **user's question**.

We only need to join the two parts together to create the prompt that we are going to send to the model.

You can limit the lenght of the context passed to the model, because we can get some Memory problems with one of the datasets that contains a realy large text in the document part.

In [17]:
question = "Can I buy a Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template

"Relevant context: #The Legendary Toshiba is Officially Done With Making Laptops #3 gaming laptop deals you can’t afford to miss today #Lenovo and HP control half of the global laptop market #Asus ROG Zephyrus G14 gaming laptop announced in India #Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865) #Apple's Next MacBook Could Be the Cheapest in Company's History #Features of Huawei's Desktop Computer Revealed #Redmi to launch its first gaming laptop on August 14: Here are all the details #Toshiba shuts the lid on laptops after 35 years #This is the cheapest Windows PC by a mile and it even has a spare SSD slot\n\n The user's question: Can I buy a Toshiba laptop?"

Now all that remains is to send the prompt to the model and wait for its response!


In [18]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Relevant context: #The Legendary Toshiba is Officially Done With Making Laptops #3 gaming laptop deals you can’t afford to miss today #Lenovo and HP control half of the global laptop market #Asus ROG Zephyrus G14 gaming laptop announced in India #Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865) #Apple's Next MacBook Could Be the Cheapest in Company's History #Features of Huawei's Desktop Computer Revealed #Redmi to launch its first gaming laptop on August 14: Here are all the details #Toshiba shuts the lid on laptops after 35 years #This is the cheapest Windows PC by a mile and it even has a spare SSD slot

 The user's question: Can I buy a Toshiba laptop?  A: Unfortunately, Toshiba has been struggling financially lately and is now officially done making laptops.




Trying another model and another question

In [19]:


model_id = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [20]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",
)

Device set to use cpu


In [21]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Relevant context: #The Legendary Toshiba is Officially Done With Making Laptops #3 gaming laptop deals you can’t afford to miss today #Lenovo and HP control half of the global laptop market #Asus ROG Zephyrus G14 gaming laptop announced in India #Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865) #Apple's Next MacBook Could Be the Cheapest in Company's History #Features of Huawei's Desktop Computer Revealed #Redmi to launch its first gaming laptop on August 14: Here are all the details #Toshiba shuts the lid on laptops after 35 years #This is the cheapest Windows PC by a mile and it even has a spare SSD slot

 The user's question: Can I buy a Toshiba laptop? 

 Please answer the relevant part of the context first, then answer the question. 

Answer:

Based on the provided context, Toshiba has officially stopped manufacturing laptops after 35 years. The article mentions "#Toshiba shuts the lid on laptops after 35 yea

In [35]:
results = collection.query(query_texts=["artifficial intelligence"], n_results=10 )

print(results)

question = "What is the latest warning regarding Artifficial Intelligence?"

context = " ".join([f"#{str(i)}" for i in results["documents"]])
#context = context[0:5120]
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template


{'ids': [['id710', 'id198', 'id2', 'id453', 'id301', 'id119', 'id407', 'id490', 'id590', 'id241']], 'embeddings': None, 'documents': [['The Ethics Of AI And Death', 'Scientists Discover New Material That Could ‘Merge AI With Human Brain’', 'Artificial intelligence warning: AI will know us better than we know ourselves', 'Ways AI could be used to facilitate crime over the next 15 years', 'Classifying galaxies with artificial intelligence', '#FossilFriday: Amazing 429 million-year-old trilobite eye saw the world like modern insects', 'IIA discovers He-enhanced cool bright stars of Omega Centauri', 'Meet FunPlus Phoenix’s new Valorant squad', 'Fossilised 429-mln-year-old eye mirrors modern insect vision', '2021 Maserati Ghibli Trofeo, Quattroporte Trofeo revealed']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'topic': 'HEALTH'}, {'topic': 'SCIENCE'}, {'topic': 'SCIENCE'}, {'topic': 'TECHNOLOGY'}, {'topic': 'SCIENCE'}, {'topic': 'SCIENC

"Relevant context: #['The Ethics Of AI And Death', 'Scientists Discover New Material That Could ‘Merge AI With Human Brain’', 'Artificial intelligence warning: AI will know us better than we know ourselves', 'Ways AI could be used to facilitate crime over the next 15 years', 'Classifying galaxies with artificial intelligence', '#FossilFriday: Amazing 429 million-year-old trilobite eye saw the world like modern insects', 'IIA discovers He-enhanced cool bright stars of Omega Centauri', 'Meet FunPlus Phoenix’s new Valorant squad', 'Fossilised 429-mln-year-old eye mirrors modern insect vision', '2021 Maserati Ghibli Trofeo, Quattroporte Trofeo revealed']\n\n The user's question: What is the latest warning regarding Artifficial Intelligence?"

In [36]:
lm_response = pipe(prompt_template)

print(lm_response[0]["generated_text"])

Relevant context: #['The Ethics Of AI And Death', 'Scientists Discover New Material That Could ‘Merge AI With Human Brain’', 'Artificial intelligence warning: AI will know us better than we know ourselves', 'Ways AI could be used to facilitate crime over the next 15 years', 'Classifying galaxies with artificial intelligence', '#FossilFriday: Amazing 429 million-year-old trilobite eye saw the world like modern insects', 'IIA discovers He-enhanced cool bright stars of Omega Centauri', 'Meet FunPlus Phoenix’s new Valorant squad', 'Fossilised 429-mln-year-old eye mirrors modern insect vision', '2021 Maserati Ghibli Trofeo, Quattroporte Trofeo revealed']

 The user's question: What is the latest warning regarding Artifficial Intelligence? What is the latest news regarding AI?

Based on the given context, the answer should be: 

The latest warning regarding artificial intelligence is from the article titled "Artificial intelligence warning: AI will know us better than we know ourselves." The